## Librerías 

In [3]:
import numpy as np
import joblib ### para cargar array

########Paquetes para NN #########
import tensorflow as tf
from sklearn import metrics ### para analizar modelo
from sklearn.ensemble import RandomForestClassifier  ### para analizar modelo
import pandas as pd

from sklearn import tree


import cv2 ### para leer imagenes jpeg
### pip install opencv-python

from matplotlib import pyplot as plt #

In [5]:
### cargar bases_procesadas ####

x_train = joblib.load('salidas/x_train.pkl')
y_train = joblib.load('salidas/y_train.pkl')
x_test = joblib.load('salidas/x_test.pkl')
y_test = joblib.load('salidas/y_test.pkl')



In [6]:
################################################
################ Preprocesamiento ##############
################################################


x_train.max()

255

In [7]:
x_test.max()

255

In [8]:
#### Escalar ######################
x_train = x_train.astype('float32') ## para poder escalarlo
x_test = x_test.astype('float32') ## para poder escalarlo
x_train /= 255 ### escalaro para que quede entre 0 y 1
x_test /= 255

###### verificar tamaños
print(x_train.shape)
x_test.shape

(2372, 100, 100, 3)


(336, 100, 100, 3)

In [9]:
np.product(x_train[1].shape) ## cantidad de variables por imagen

30000

In [10]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([1569,  803]))

In [11]:
np.unique(y_test, return_counts=True)

(array([0, 1]), array([208, 128]))

In [12]:
##### convertir a 1 d array ############
x_train2 = x_train.reshape(2372,30000)
x_test2 = x_test.reshape(336, 30000)
print(x_train2.shape)
x_test2.shape

(2372, 30000)


(336, 30000)

## Selección de métricas 

### AUC = 

## 1. Modelo Random Forest

In [17]:
rf=RandomForestClassifier()
rf.fit(x_train2, y_train)

pred_train=rf.predict(x_train2)
r_metric=metrics.recall_score(y_train, pred_train)
auc_metric=metrics.roc_auc_score(y_train, pred_train)

metricas_train = pd.DataFrame({
    'Metric': ['Recall', 'AUC'],
    'Score': [r_metric, auc_metric]
})

print(metricas_train)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


   Metric  Score
0  Recall    1.0
1     AUC    1.0


In [18]:
pred_test=rf.predict(x_test2)
r_metric=metrics.recall_score(y_test, pred_test)
auc_metric=metrics.roc_auc_score(y_test, pred_test)

metricas_test = pd.DataFrame({
    'Metric': ['Recall', 'AUC'],
    'Score': [r_metric, auc_metric]
})

print(metricas_test)

   Metric     Score
0  Recall  0.078125
1     AUC  0.519832
